In [2]:
from dotenv import find_dotenv, load_dotenv
import openai
import os 


_ = load_dotenv(find_dotenv())
openai.api_key = os.environ.get("OPENAI_API_KEY")

# Load Brand Docs

In [4]:
with open(os.path.join(os.getcwd(), "test_data", "brand_doc1.txt"), 'r') as f:
    brand_doc1 = f.read()

with open(os.path.join(os.getcwd(), "test_data", "brand_doc2.txt"), 'r') as f:
    brand_doc2 = f.read()

In [10]:
concatenated = "\n\n---\n\n".join([brand_doc1, brand_doc2])

In [11]:
print(concatenated)

This is the 1st brand doc.

---

And this is the second brand doc.


# Process

In [14]:
from langchain_core.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_openai import ChatOpenAI
from langchain_core.pydantic_v1 import BaseModel, Field

In [22]:
model = ChatOpenAI(model="gpt-4-turbo", temperature=0)

In [29]:
system_message = """You are an expert in branding and marketing. 
You are working with a team of copywriters and you help them understand the brand, its identity, concepts, culture and heritage. 
You have received several documents about the brand. 
Your task is to condense those documents into a concise branding brief that will help and guide the copywriters produce on brand content.
"""

human_message = """Here are the brand documents separated by ---. Please condense them into a concise brand brief.
<brand_documents>
{brand_documents}
</brand_documents>

{format_instructions}
"""


In [24]:
class BrandKnowledge(BaseModel):
    brand_knowledge: str = Field(description="The condensed brand knowledge extracted from the documents to help copywriters.")

output_parser = JsonOutputParser(pydantic_object=BrandKnowledge)


In [25]:
prompt = ChatPromptTemplate(
    messages=[
        SystemMessagePromptTemplate.from_template(system_message),
        HumanMessagePromptTemplate.from_template(human_message),
    ],
    input_variables=["brand_documents"],
    partial_variables={'format_instructions': output_parser.get_format_instructions()}
)

In [27]:
brand_knowledge_extraction_chain = prompt | model | output_parser

In [28]:
brand_knowledge_extraction_chain.invoke({"brand_documents": concatenated})

{'brand_knowledge': 'This is the 1st brand doc. And this is the second brand doc.'}

---

# Copywriting Guidelines Extraction

In [30]:
system_message = """You are a journalist expert in writing on brand text. 
You are working with a team of copywriters and you role is to guide them to produce perfect copies by providing them an understandable set of copywriting guidelines based on the provided documents.
You have received several copywriting guidelines related documents . 
Your task is to condense those documents into concise copywriting guidelines that will help and guide the copywriters produce perfect content.
"""

human_message = """Here are the copywriting documents separated by ---. Please condense them into concise copywriting guidelines.
<copywriting_guidelines_documents>
{copywriting_guidelines_documents}
</copywriting_guidelines_documents>

{format_instructions}
"""


In [31]:
class CopywritingGuidelines(BaseModel):
    copywriting_guidelines: str = Field(description="The condensed copywriting guidelines extracted from the documents to help copywriters.")

output_parser = JsonOutputParser(pydantic_object=CopywritingGuidelines)

In [32]:
prompt = ChatPromptTemplate(
    messages=[
        SystemMessagePromptTemplate.from_template(system_message),
        HumanMessagePromptTemplate.from_template(human_message),
    ],
    input_variables=["copywriting_guidelines_documents"],
    partial_variables={'format_instructions': output_parser.get_format_instructions()}
)


In [33]:
copywriting_guidelines_extraction_chain = prompt | model | output_parser

In [34]:
copywriting_guidelines_extraction_chain.invoke({"copywriting_guidelines_documents": concatenated})

{'copywriting_guidelines': 'This is the 1st brand doc. And this is the second brand doc.'}